In [2]:
import pandas as pd

train_df = pd.read_csv('./data/train.csv')
val_df = pd.read_csv('./data/val.csv')
test_df = pd.read_csv('./data/test.csv')

In [ ]:
corr = val_df.corr()
abs(corr['Class']).sort_values()

In [4]:
val_X = val_df.drop(columns = ['ID','Class'],axis = 1)
val_y = val_df[['Class']]

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
pca_transformed = pca.fit_transform(train_df)

train_df['pca_train_x'] = pca_transformed[:, 0]
train_df['pca_train_y'] = pca_transformed[:, 1]

train_df

In [ ]:
train_df = train_df.drop(columns = ['ID'],axis = 1)
train_df

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, normalize
sc= StandardScaler()  # using standard scaler
train_scaled = sc.fit_transform(train_df)
val_scaled = sc.fit_transform(val_X)
train_scaled

In [8]:
train_copy = train_scaled.copy()

In [5]:
from sklearn.cluster import DBSCAN

In [11]:
def get_outlier(df, column, weight = 1.5):
    # 25분위와 75분위 계산
    q25 = train_copy.quantile(0.25)
    q75 = train_copy.quantile(0.75)
    iqr = q75 - q25
    lowest = q25 - iqr * weight
    highest = q75 + iqr * weight
    return fraud[(fraud > highest) | (fraud < lowest)].index

In [ ]:
get_outlier(train_copy, 'V17', 1)

In [ ]:
dbscan = DBSCAN(eps = 0.5, min_samples = 15, metric = 'euclidean', n_jobs = -1)
dbscan_train = dbscan.fit(train_copy)
dbscan_labels = dbscan.fit_predict(train_copy)

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=11, prediction_data=True).fit(train_scaled)
clusterer

In [10]:
hdbscan_labels, strengths = hdbscan.approximate_predict(clusterer, train_scaled)

In [ ]:
pd.DataFrame(hdbscan_labels).value_counts()